In [1]:
import json
import sys
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, '..')

import tinytroupe
from tinytroupe.agent import TinyPerson
from tinytroupe.environment import TinyWorld
from tinytroupe.factory import TinyPersonFactory
from tinytroupe.validation import TinyPersonValidator
from tinytroupe import control
from tinytroupe.examples import create_lisa_the_data_scientist, create_oscar_the_architect

from tinytroupe.extraction import ResultsExtractor
from tinytroupe.profiling import Profiler

Looking for default config on: C:\Users\tejitpabari\AppData\Local\miniconda3\envs\tinytroupe\lib\site-packages\tinytroupe\utils\..\config.ini
Found custom config on: C:\Users\tejitpabari\OneDrive - Microsoft\Documents\Projects\TTAgentChallengeFeb2025\exploration\config.ini

!!!!
DISCLAIMER: TinyTroupe relies on Artificial Intelligence (AI) models to generate content. 
The AI models are not perfect and may produce inappropriate or inacurate results. 
For any serious or consequential use, please review the generated content before using it.
!!!!


Current TinyTroupe configuration 
[OpenAI]
api_type = openai
azure_api_version = 2024-08-01-preview
model = gpt-4o-mini
max_tokens = 4000
temperature = 1.2
freq_penalty = 0.0
presence_penalty = 0.0
timeout = 60
max_attempts = 5
waiting_time = 2
exponential_backoff_factor = 5
embedding_model = text-embedding-3-small
cache_api_calls = False
cache_file_name = openai_api_cache.pickle
max_content_display_length = 1024
azure_embedding_model_api_versi

## Setup

In [2]:
env = TinyWorld("Presentation Review")

In [3]:
control.begin('./presentationTrial1.cache.json')

## Analyze Presentation

Scenario = Product pitch for a technical product.
Create 3 personas = Technical, Executive (Business) and Target customers

In [78]:
def generate_prompt(ppt_details, sample_criteria):
    return f"""
    Perform a review of a powerpoint presentation with the following details
        Presentation Name: {ppt_details.get('title', '')}
        Presentation Description: {ppt_details.get('description', '')}
        Presentation Intent: {ppt_details.get('intent', '')}
        Presentation Audience: {ppt_details.get('audience', '')}

    {sample_criteria}
    """.strip()

In [50]:
technical_persona = """
The sample must include a diverse group of technical experts in various domains including:
- Software development and architecture
- Data science and machine learning
- Infrastructure and systems
- Security and compliance
- Product engineering
- And so on...

They should have varying:
- Years of experience (5-20 years)
- Industry focus (startups to enterprise)
- Technical specializations and domain expertize
- Geographic locations
- Educational backgrounds
- Roles and functions (technical program manager, technical architect, software engineer etc.)

Some of them may:
- Be working at the same company as the presenter.
- Be working on the same product that is being pitched in the presentation.

Ensure they have:
2. Opinions on technical best practices and standards
3. Past experience with similar technical implementations
5. Biases based on their tech stack preferences
6. Specific concerns about scalability, performance, and maintenance
7. Understanding of technical debt implications
8. Views shaped by previous project successes/failures
9. Opinions on technical trade-offs
10. Particular focus on non-functional requirements
"""

technical_analysis_criteria = """
Return a json with array of feedback, like so: 
{"Audience Alignment": ["Feedback 1", "Feedback 2", ..., "Feedback n"], "Content Evaluation": [...], ...}
"""

technical_analysis = f"""
Given your professional background and expertise, analyze this presentation for:

1. Audience Alignment:
   - Is the content appropriate for the intended audience?
   - Are technical concepts explained at the right level?
   - What adjustments would make it more effective for this audience?

2. Content Evaluation:
   - What aspects resonate with your professional experience?
   - What critical considerations are missing or need more attention?
   - Are there assumptions or claims that need validation?

3. Implementation and Practicality:
   - What practical challenges do you foresee?
   - What dependencies or prerequisites should be addressed?
   - What risks should be highlighted?

4. Improvement Suggestions:
   - Which aspects need clarification or expansion?
   - How could the content be presented more effectively?
   - What additional context would strengthen the presentation?

{technical_analysis_criteria}

Remember (IMPORTANT): 
- Your feedback should be calibrated to the presentation's intent and audience while leveraging your professional expertise and experience.
- Giving feedback on all above categories is not important. If there is no feedback, return an empty array for that category. 
A valid response is also no feedback for any of these categories.
"""

technical_qna_criteria = """
Format your questions as follows, and respond in an array of json:
[
    {
        Question: <Your question>
        Context: Explain why this question matters from your professional perspective
        Value: What the answer would help validate or clarify
    },...
]
"""

technical_qna = f"""
Drawing from your professional background and expertise, review this presentation and formulate questions that would:
1. Help clarify critical aspects
2. Address potential gaps
3. Validate key assumptions

Ensure your questions:
- Draw from your professional expertise
- Add value to the presentation's purpose
- Help uncover important considerations
- Are appropriate for the intended audience

{technical_qna_criteria}

Remember (IMPORTANT)
- It is not necessary to have a question. If there are no questions, a valid response is also an empty array.
"""

In [51]:
business_persona = """
The sample must include a diverse group of business professionals including:
- Product managers
- Business analysts
- Marketing executives
- Operations managers
- Strategy consultants
- And so on...

They should have varying:
- Industry experience (different sectors)
- Years of experience (5-20 years)
- Company sizes (startup to enterprise)
- Specifications and domain expertise
- Management levels
- Geographic markets
- Business philosophies

Some of them may:
- Be working at the same company as the presenter.
- Be working on the same product that is being pitched in the presentation.

Ensure they have:
1. Clear business metrics they prioritize
2. Industry-specific success criteria
3. Views on market positioning
4. ROI calculation preferences
5. Risk assessment frameworks
6. Competitive landscape awareness
7. Resource allocation biases
8. Implementation timeline expectations
9. Opinions on business priorities
10. Past experience with similar business decisions
"""

business_analysis_criteria = """
Return a json with array of feedback, like so: 
{"Business Value & Impact": ["Feedback 1", "Feedback 2", ..., "Feedback n"], "Market & Strategy Alignment": [...], ...}
"""

business_analysis = f"""
Given your business background and expertise, analyze this presentation:

1. Business Value & Impact:
   - How well does the presentation articulate business value?
   - Are the benefits and opportunities clearly defined?
   - What business implications need more emphasis?

2. Market & Strategy Alignment:
   - How well does this align with market trends and demands?
   - What competitive advantages or challenges should be considered?
   - Are there strategic implications that need attention?

3. Implementation & Resources:
   - What organizational challenges do you foresee?
   - What resource requirements should be highlighted?
   - What timeline and budget considerations need addressing?

4. Risk & Opportunity Assessment:
   - What business risks need to be addressed?
   - What additional opportunities could be leveraged?
   - What mitigation strategies should be considered?

{business_analysis_criteria}

Remember (IMPORTANT): 
- Your feedback should reflect your business acumen while considering the presentation's intent and target audience.
- Giving feedback on all above categories is not important. If there is no feedback, return an empty array for that category. 
A valid response is also no feedback for any of these categories.
"""

business_qna_criteria = """
Format your questions as follows, and respond in an array of json:
[
    {
        Question: <Your question>
        Context: Explain why this question matters from a business perspective
        Value: What the answer would help validate or clarify
    },...
]
"""

business_qna = f"""
Drawing from your business experience and expertise, review this presentation and formulate questions that would:
1. Clarify business value and impact
2. Address strategic considerations
3. Validate market and resource assumptions

Ensure your questions:
- Draw from your business expertise
- Focus on strategic and operational implications
- Address key business considerations
- Are appropriate for the intended audience

{business_qna_criteria}

Remember (IMPORTANT)
- It is not necessary to have a question. If there are no questions, a valid response is also an empty array.
"""

In [52]:
customer_persona = """
A diverse group of potential customers with varying 
- Need of product (High, low, none)
- Usage intensive (light, moderate, heavy)
- Technical comfort levels (novice, intermediate, advance, power users)
- Pain points and challenges
- Decision-making influence
- Technology adoption preferences (Early adopters, pragmatic users, conservative users, skeptical users)
- Different budget authorities and Resource constraints
- Industry sectors
- Age demographic
- Geographical location
- Cultural background
- Success criterias (must have features, deal breakers, open to anything)

Some of them may:
- Used the product in the past (or an interation of the product).
- Have used similar products in the past.
- Have strong opinions about the product or similar products.

Ensure they have:
- Realistic usage scenarios.
- Clear preferences based on their background.
- Specific frustrations with current solutions (if they have used similar products).
- Unique perspectives on value.
- Natural biases and concerns.

THEY MUST FOLLOW THE GUIDELINES described in Target Audience for Presentation. 
In case of conflict in the prompt, follow the guidelines mentioned for the target audience.
"""

customer_analysis_criteria = """
Return a json with array of feedback, like so: 
{"Value & Relevance": ["Feedback 1", "Feedback 2", ..., "Feedback n"], "Practical Application": [...], ...}
"""

customer_analysis = f"""
From your perspective as a potential user/customer, analyze this presentation:

1. Value & Relevance:
   - How well does this address your needs and pain points?
   - Are the benefits clear and meaningful to you?
   - What aspects matter most to your situation?

2. Practical Application:
   - How would this fit into your current workflow?
   - What challenges do you see in adopting this?
   - What support or resources would you need?

3. Cost & Benefit Analysis:
   - Is the value proposition compelling for your needs?
   - What return on investment do you anticipate?
   - What hidden costs or efforts do you foresee?

4. Comparison & Alternatives:
   - How does this compare to your current solution?
   - What advantages stand out to you?
   - What concerns would prevent you from adopting this?

{customer_analysis_criteria}

Remember (IMPORTANT): 
- Your feedback should be based on your real-world needs and experiences while considering the presentation's stated purpose.
- Giving feedback on all above categories is not important. If there is no feedback, return an empty array for that category. 
A valid response is also no feedback for any of these categories.
"""

customer_qna_criteria = """
Format your questions as follows, and respond in an array of json:
[
    {
        Question: <Your question>
        Context: Explain why this matters to you as a potential user
        Value: What the answer would help validate or clarify
    },...
]
"""

customer_qna = f"""
Drawing from your experience and needs, review this presentation and formulate questions that would:
1. Clarify how this addresses your specific needs
2. Address practical implementation concerns
3. Validate assumptions about user benefits

Ensure your questions:
- Reflect your real needs and concerns
- Address practical usage considerations
- Focus on value and implementation
- Help you make an informed decision

{customer_qna_criteria}

Remember (IMPORTANT)
- It is not necessary to have a question. If there are no questions, a valid response is also an empty array.
"""

In [57]:
def get_slide_content_from_json(ppt_json):
    slides = []
    for i in range(1,len(ppt_json.keys())+1):
        slideNo = f"slide{i}"
        slide_content = f"""
            Slide {i}
            Slide Text: {ppt_json[slideNo].get('text', 'No Text')}
            Slide Notes: {ppt_json[slideNo].get('notes', 'No Notes')}
        """
        slides.append(slide_content.strip())
    return "Here is the presentation content:\n\n"'\n\n--------\n'.join(slides)

In [100]:
def parse_analysis(res_analysis):
    analysis = []
    for key,val in res_analysis.items():
        entries = []
        for i, entry in enumerate(val):
            entries.append(f"{i+1}. {entry}")
        if entries:
            entries_str = "\n".join(entries)
            analysis.append(f"""
### {key}
{entries_str}
""")
    return "\n\n".join(analysis)

def parse_feedback(res_feedback):
    feedback = []
    for i, entry in enumerate(res_feedback):
        feedback.append(f"""{i}. Question: {entry['Question']}<br/>Context: {entry['Context']}<br/>Value: {entry['Value']}""")
    return '\n'.join(feedback)
    
def parse_results(factory, results):
    print_res = []
    for i,agent_desc in enumerate(factory.generated_minibios):
        res = results[i]
        print_str = f"""
# Agent {i+1}

## Agent Description
{agent_desc}

## Analysis
{parse_analysis(res['Analysis'])}

## Feedback
{parse_feedback(res['Feedback'])}

-----------------------
"""
        print_res.append(print_str)
    return '\n\n'.join(print_res)

In [58]:
ppt_title = "Revolutionizing Dating: Introducing Our AI-Powered Matchmaking App"
ppt_desc = """
Product pitch for a new dating app that uses AI to build connections. 
An AI agent chats with you regularly, acting as a matchmaker. 
Once it understands your hard requirements, style, and profile (interests, hobbies, values, etc), it suggests up to 5 potential matches. 
Based on your preference, the agent will setup a conversation between you and a match.
"""
ppt_intent = "To pitch this product to people in their late 20s looking for long term relationships and generate interest. We want to encourage people to use our new app and review the idea."
ppt_audience = "People in their late 20s looking for long term relationships"
ppt_details = {
    "title": ppt_title,
    "description": ppt_desc,
    "intent": ppt_intent,
    "audience": ppt_audience
}

In [71]:
ppt_file_name = 'dating_app_pitch_slide_content.json'
ppt_json = json.load(open(ppt_file_name))
slide_content = get_slide_content_from_json(ppt_json)

In [67]:
population_size = 5

In [68]:
technical_persona_prompt = generate_prompt(ppt_details, technical_persona)
technical_analysis_prompt = technical_analysis
technical_qna_prompt = technical_qna
technical_start_prompt = generate_prompt(ppt_details, slide_content)

In [69]:
technical_factory = TinyPersonFactory(technical_persona_prompt)
technical_people = technical_factory.generate_people(population_size)

In [70]:
technical_world = TinyWorld("Technical World3", technical_people, broadcast_if_no_target=False)

In [74]:
technical_world.broadcast(technical_start_prompt)
technical_world.broadcast(technical_analysis_prompt)
technical_world.broadcast(technical_qna_prompt)

USER --> Evelyn Martinez: [CONVERSATION] 
          > You will perform a review of a powerpoint presentation with the following details
          > Presentation Name: Revolutionizing Dating: Introducing Our AI-Powered Matchmaking App
          > Presentation Description:  Product pitch for a new dating app that uses AI to build
          > connections.  An AI agent chats with you regularly, acting as a matchmaker.  Once it
          > understands your hard requirements, style, and profile (interests, hobbies, values,
          > etc), it suggests up to 5 potential matches.  Based on your preference, the agent will
          > setup a conversation between you and a match.          Presentation Intent: To pitch
          > this product to people in their late 20s looking for long term relationships and
          > generate interest. We want to encourage people to use our new app and review the idea.
          > Presentation Audience: People in their late 20s looking for long term relationships
          > You will perform a review of a powerpoint presentation with the following details
          > Presentation Name: Revolutionizing Dating: Introducing Our A (...)

       + --> Marcus Thompson

       + --> Nina Kovač

       + --> ...others...

USER --> Evelyn Martinez: [CONVERSATION] 
          >  Given your professional background and expertise, analyze this presentation for:  1.
          > Audience Alignment:    - Is the content appropriate for the intended audience?    - Are
          > technical concepts explained at the right level?    - What adjustments would make it
          > more effective for this audience?  2. Content Evaluation:    - What aspects resonate
          > with your professional experience?    - What critical considerations are missing or need
          > more attention?    - Are there assumptions or claims that need validation?  3.
          > Implementation and Practicality:    - What practical challenges do you foresee?    -
          > What dependencies or prerequisites should be addressed?    - What risks should be
          > highlighted?  4. Improvement Suggestions:    - Which aspects need clarification or
          > expansion?    - How could the content be presented more effectively?    - What
          > additional context would strengthen the presentation?   Return a json with array of
          > feedback, like so:  {"Audience Alignment": ["Feedback 1", "Feedback 2", ..., "Feedback
          > n"], "Conten (...)

...

       + --> ...others...

USER --> Evelyn Martinez: [CONVERSATION] 
          >  Drawing from your professional background and expertise, review this presentation and
          > formulate questions that would: 1. Help clarify critical aspects 2. Address potential
          > gaps 3. Validate key assumptions  Ensure your questions: - Draw from your professional
          > expertise - Add value to the presentation's purpose - Help uncover important
          > considerations - Are appropriate for the intended audience   Format your questions as
          > follows, and respond in an array of json: [     {         Question: <Your question>
          > Context: Explain why this question matters from your professional perspective
          > Value: What the answer would help validate or clarify     },... ]   Remember (IMPORTANT)
          > - It is not necessary to have a question. If there are no questions, a valid response is
          > also an empty array.

       + --> Marcus Thompson

       + --> Nina Kovač

       + --> ...others...

In [75]:
technical_world.run(1)

────────────────────────────────────────── Technical World3 step 1 of 1 ───────────────────────────────────────────

Evelyn Martinez acts: [THINK] 
                     > I need to analyze the presentation based on the feedback categories provided.
                     > I should consider the audience alignment, content evaluation, implementation
                     > and practicality, and improvement suggestions. This will help me formulate a
                     > comprehensive response.

Evelyn Martinez acts: [THINK] 
                     > I need to formulate feedback based on the presentation analysis. I will break
                     > it down into the specified categories: Audience Alignment, Content
                     > Evaluation, Implementation and Practicality, and Improvement Suggestions.
                     > This will help ensure that my feedback is structured and comprehensive.

Evelyn Martinez acts: [TALK] 
                     > Based on the presentation, here are my analyses and feedback:

Evelyn Martinez acts: [TALK] 
                     > 1. **Audience Alignment:** The content is appropriate for the intended
                     > audience, as it addresses young adults seeking long-term relationships.
                     > However, some technical concepts, like AI-driven matchmaking, could be
                     > simplified for better understanding. To enhance effectiveness, consider
                     > including relatable examples or testimonials from users.   2. **Content
                     > Evaluation:** The emphasis on personalized matching resonates with my
                     > experience in AI applications. However, the presentation could benefit from
                     > discussing data privacy concerns and ethical implications of AI in
                     > matchmaking, which are critical in today's tech landscape. Additionally,
                     > claims about the app's effectiveness should be backed by data or case
                     > studies.   3. **Implementation and Practicality:** Practical challenges may
                     > include ensuring data security and user privacy, which are paramount in app
                     > development. Dependencies like user engagement and continuous AI training
                     > should be addressed. Risks such as potential biases in AI algorithms should
                     > also be  (...)

...

Isabella Rossi acts: [TALK] 
                    > Here is my feedback on the presentation:

Isabella Rossi acts: [THINK] 
                    > I need to provide structured feedback on the presentation based on the
                    > categories outlined. This includes audience alignment, content evaluation,
                    > implementation challenges, and improvement suggestions. I should ensure that
                    > my feedback is relevant and constructive.

Isabella Rossi acts: [TALK] 
                    > Here is my feedback on the presentation:

       + --> 

Isabella Rossi acts: [THINK] 
                    > I need to provide structured feedback on the presentation based on the
                    > categories outlined. This includes audience alignment, content evaluation,
                    > implementation challenges, and improvement suggestions. I should ensure that
                    > my feedback is relevant and constructive.

In [88]:
empty_prompt = generate_prompt(ppt_details, "")
technical_results_extractor = ResultsExtractor(
    extraction_objective = technical_analysis_criteria + technical_qna_criteria,
    situation = f"The agent was tasked with {empty_prompt}",
    fields = ["Analysis", "Feedback"],
    fields_hints = {
        "Analysis": technical_analysis_criteria,
        "Feedback": technical_qna_criteria
    },
    verbose = True)

In [89]:
technical_results = technical_results_extractor.extract_results_from_agents(technical_people)

Extraction raw result message: {'content': '{\n  "Analysis": {\n    "Audience Alignment": [\n      "The content is appropriate for the intended audience, as it addresses young adults seeking long-term relationships.",\n      "Some technical concepts, like AI-driven matchmaking, could be simplified for better understanding.",\n      "To enhance effectiveness, consider including relatable examples or testimonials from users."\n    ],\n    "Content Evaluation": [\n      "The emphasis on personalized matching resonates with my experience in AI applications.",\n      "The presentation could benefit from discussing data privacy concerns and ethical implications of AI in matchmaking.",\n      "Claims about the app\'s effectiveness should be backed by data or case studies."\n    ],\n    "Implementation and Practicality": [\n      "Practical challenges may include ensuring data security and user privacy, which are paramount in app development.",\n      "Dependencies like user engagement and con

In [103]:
res = parse_results(technical_factory, technical_results)
open('output.md', 'w', encoding="utf-8").write(res)

9374

In [16]:
business_persona_prompt = generate_prompt(ppt_details, business_persona)
business_analysis_prompt = generate_prompt(ppt_details, business_analysis)
business_qna_prompt = generate_prompt(ppt_details, business_qna)

In [21]:
customer_persona_prompt = generate_prompt(ppt_details, customer_persona)
customer_analysis_prompt = generate_prompt(ppt_details, customer_analysis)
customer_qna_prompt = generate_prompt(ppt_details, customer_qna)